# 1. Modellevaluierung für Option 1, 2 and 3

In [1]:
pip install --upgrade --force-reinstall gensim

  Using cached gensim-4.3.1-cp39-cp39-win_amd64.whl (24.0 MB)
  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)
  Using cached scipy-1.10.1-cp39-cp39-win_amd64.whl (42.5 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.3.0
    Uninstalling smart-open-6.3.0:
      Successfully uninstalled smart-open-6.3.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade --force-reinstall numpy

  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
# from google.colab import files
 
 
# uploaded = files.upload()

In [4]:
import pandas as pd

In [5]:
def helper(column):
    return str(column).split(",")

def open_csv_default():
    df_data = pd.read_csv("naukri_data_science_jobs_india.csv")
    df_data = df_data.drop(columns=['Company', 'Location', 'Job Experience'])
    df_data["Skills/Description"] =df_data["Skills/Description"].apply(lambda x : helper(x))
    return df_data

'''
def open_csv_clean():
    df_data = pd.read_csv("naukri_data_science_jobs_india_cleaned.csv")
    #df_data = df_data.drop(columns=['Company', 'Location', 'Job Experience'])
    #df_data["Skills/Description"] =df_data["Skills/Description"].apply(lambda x : helper(x))
    return df_data
'''

def open_csv_clean():
    df_data = pd.read_csv("naukri_data_science_jobs_india_cleaned.csv")
    formatted = df_data.values.tolist()
    return formatted

def open_csv_clustered():
    df_data = pd.read_csv("naukri_data_science_jobs_india_cleaned_clusterd.csv")
    df_data = df_data.drop(columns=['Job_Role', 'lda_score'])
    df_data = df_data[['lda_topic', 'Skills/Description']]
    return df_data

In [6]:
'''
### Option 1: Use Doc2Vec which includes the "document title" in learning.
## Works out of the box

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import ast

#data = open_csv_default()
data = open_csv_clean()

# Preprocess the data and create TaggedDocuments
#tagged_data = [TaggedDocument(words=skills, tags = [job]) for skills in data]
#tagged_data = [TaggedDocument(words=skills, tags=[job]) for job, skills in data]
tagged_data = [TaggedDocument(words=ast.literal_eval(skills), tags=[job]) for job, skills in data]

# Train the Doc2Vec model
model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model.build_vocab(tagged_data)

model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
'''

'\n### Option 1: Use Doc2Vec which includes the "document title" in learning.\n## Works out of the box\n\nfrom gensim.models import Doc2Vec\nfrom gensim.models.doc2vec import TaggedDocument\nimport ast\n\n#data = open_csv_default()\ndata = open_csv_clean()\n\n# Preprocess the data and create TaggedDocuments\n#tagged_data = [TaggedDocument(words=skills, tags = [job]) for skills in data]\n#tagged_data = [TaggedDocument(words=skills, tags=[job]) for job, skills in data]\ntagged_data = [TaggedDocument(words=ast.literal_eval(skills), tags=[job]) for job, skills in data]\n\n# Train the Doc2Vec model\nmodel = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)\nmodel.build_vocab(tagged_data)\n\nmodel.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)\n'

In [7]:
'''
import numpy

# Test the model
#print(model.wv["python"])
print(numpy.linalg.norm(model.wv["advanced_analytics"] - model.wv["analytics"]))
print(numpy.linalg.norm(model.wv["machine_learning"] - model.wv["management"]))

# Enter skills and get most suitable jobs
skills = ['machine_learning', 'advanced_analytics']
infer_vector = model.infer_vector(skills)
similar_jobs = model.dv.most_similar([infer_vector], topn=5)

print(f"Given skills: {skills}")
print("Most suitable jobs:")
for job, similarity in similar_jobs:
    print(f"- {job} (Similarity: {similarity:.2f})")
'''

'\nimport numpy\n\n# Test the model\n#print(model.wv["python"])\nprint(numpy.linalg.norm(model.wv["advanced_analytics"] - model.wv["analytics"]))\nprint(numpy.linalg.norm(model.wv["machine_learning"] - model.wv["management"]))\n\n# Enter skills and get most suitable jobs\nskills = [\'machine_learning\', \'advanced_analytics\']\ninfer_vector = model.infer_vector(skills)\nsimilar_jobs = model.dv.most_similar([infer_vector], topn=5)\n\nprint(f"Given skills: {skills}")\nprint("Most suitable jobs:")\nfor job, similarity in similar_jobs:\n    print(f"- {job} (Similarity: {similarity:.2f})")\n'

In [8]:
### Training uses the Doc2Vec or Word2Vec model which simply embeds words according to how often they occur together (ignoring order afaik)
## Adapt vector size and epochs in training once data set is complete

from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import ast
import numpy

def trainWord2Vec(data):
# Preprocess the data and create list of skills (this training does NOT include jobs!)
    tagged_data = [ast.literal_eval(skills) for job, skills in data]

# Train the Word2Vec model, try different vector sizes for interesting effects in similarities
    model = Word2Vec(vector_size=50, min_count=1, workers=4, epochs=20)
    model.build_vocab(tagged_data)

    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

    return model

def trainDoc2Vec(data):
    tagged_data = [TaggedDocument(words=ast.literal_eval(skills), tags=[job]) for job, skills in data[:]]

# Train the Doc2Vec model
    model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
    model.build_vocab(tagged_data)

    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

    return model

# helper method to calculate average embedding vector of a list of string skills, filters out skills unknown to the model
def averageOfSkills(model, input, axis = 0):
    vectors = [model.wv[i] for i in input if model.wv.has_index_for(i)]
    if(len(vectors) == 0):
        return numpy.zeros(len(model.wv[0]))
    
    return numpy.average(vectors, axis = axis)

In [9]:
import numpy

def calcAvgEmbeddings(model, data):

    # Tests for the embedding go here
    print(model.wv.most_similar("neural_networks", topn = 5))

    #cosine distance
    print(model.wv.distance("finance", "machine_learning"))
    print(model.wv.distance("ai", "machine_learning"))
    print(model.wv.distance("business_intelligence", "management"))
    print(model.wv.distance('corporate_governance', 'ai'))
    #euclidean distance
    #print(numpy.linalg.norm(model.wv["statistics"] - model.wv["machine_learning"]))

    # split the tuples
    job_titles, skills = list(zip(*data))
    # calculate the average skill vector of every job offering / person and zip back together
    skillAverages = [averageOfSkills(model, ast.literal_eval(skill)) for skill in skills]
    data_averaged = list(zip(job_titles, skillAverages))

    return (skillAverages, data_averaged)

# This gives us a list of job offerings and average embeddings. 
# Could be used as input to a graph neural network or knowledge graph?
# Is used below to classify immediately

In [10]:
# simply suggest smallest distance to user's average ("Option 2") using word2Vec
import itertools
import operator
from scipy.spatial.distance import cdist

def prepareOption2(data):

    model = trainWord2Vec(data)
    skillAverages, data_averaged = calcAvgEmbeddings(model, data)

    # sort before grouping
    data_averaged.sort(key=operator.itemgetter(0))
    # Group by job title, take the average embedding of everyone with that title and make a dictionary (maps job title to average embedding)
    job_averages = {key : numpy.average(list(zip(*list(value)))[1], axis = 0)
        for key, value in itertools.groupby(data_averaged, lambda x: x[0])}

    #keep keys and values
    keys = list(job_averages.keys())
    values = list(job_averages.values())
    # easy access to the avg vector
    #print(job_averages['Advisor, Data Science'])
    
    # model.save("word2vec_model_option2.bin")

    return job_averages, keys, values, model

# usage in next cells

In [11]:
# "Option 3" could be using the embedding learned by Doc2Vec and doing the same manual averaging as Option 2 for "learning" the correlation of job to skills
import itertools
import operator
from scipy.spatial.distance import cdist

def prepareOption3(data):
#if !data:
#  data = open_csv_clean()

    model = trainDoc2Vec(data)
    skillAverages, data_averaged = calcAvgEmbeddings(model, data)

    # sort before grouping
    data_averaged.sort(key=operator.itemgetter(0))
    # Group by job title, take the average embedding of everyone with that title and make a dictionary (maps job title to average embedding)
    job_averages = {key : numpy.average(list(zip(*list(value)))[1], axis = 0)
        for key, value in itertools.groupby(data_averaged, lambda x: x[0])}

    #keep keys and values
    keys = list(job_averages.keys())
    values = list(job_averages.values())
    
    # model.save("doc2vec_model_option3.bin")

    return job_averages, keys, values, model

In [12]:
'''
# USAGE of Option 2 and 3:
# (not a proper test yet)


# data = open_csv_clean()
# formatted = data.values.tolist()
formatted = open_csv_clean()



job_averages, keys, values, model = prepareOption3(data)
#job_averages, keys, values, model = prepareOption2(data)

#Given some skill list:
#my_skills = ["business_intelligence", "management"]
my_skills = ['python', 'tableau', 'machine_learning', 'sql', 'statistical_modeling', 'adv_excel', 'microstrategy', 'r_shiny']
# we can calculate the average embedding
avg = averageOfSkills(model, my_skills)

#and match it to the closest job title
# calculate distance to all jobs for lack of better solution, optimize this if there's time
distVec = cdist([avg], values)
# zip the distances with the keys and sort jobs by distance
sortedList = sorted(list(zip(distVec.transpose(), keys)))
# print top 5 scoring jobs (i.e. smallest distance)
result = [("" + str(i) +": Score " + str(1 / max(scor[0], 0.1))) for scor,i in sortedList[0:5]]
#result = [("" + str(i) +": Score " + str(scor)) for scor,i in sortedList[0:5]]
print("Top Jobs: ")
for i in range(5):
    print(result[i])

# Our model option 2 should immediately support another application: Find top 10 skills of some job

job = "Data Manager"

embed = job_averages[job]

embed = embed / numpy.linalg.norm(embed)

distVec = cdist([embed], model.wv.get_normed_vectors())

print(distVec.shape)
print(len(distVec))

sortIndicesByDist = sorted(list(zip(distVec.transpose(), range(len(distVec.transpose())))))
result = [(model.wv.index_to_key[index], score) for score,index in sortIndicesByDist[0:10]]

print("Main skills for "+ job)
for i in range(10):
    print(result[i])
'''

'\n# USAGE of Option 2 and 3:\n# (not a proper test yet)\n\n\n# data = open_csv_clean()\n# formatted = data.values.tolist()\nformatted = open_csv_clean()\n\n\n\njob_averages, keys, values, model = prepareOption3(data)\n#job_averages, keys, values, model = prepareOption2(data)\n\n#Given some skill list:\n#my_skills = ["business_intelligence", "management"]\nmy_skills = [\'python\', \'tableau\', \'machine_learning\', \'sql\', \'statistical_modeling\', \'adv_excel\', \'microstrategy\', \'r_shiny\']\n# we can calculate the average embedding\navg = averageOfSkills(model, my_skills)\n\n#and match it to the closest job title\n# calculate distance to all jobs for lack of better solution, optimize this if there\'s time\ndistVec = cdist([avg], values)\n# zip the distances with the keys and sort jobs by distance\nsortedList = sorted(list(zip(distVec.transpose(), keys)))\n# print top 5 scoring jobs (i.e. smallest distance)\nresult = [("" + str(i) +": Score " + str(1 / max(scor[0], 0.1))) for scor,

In [13]:
'''
# Proper testing
import heapq

# divide data into 75 train, 25 test
def train_test_split(df, frac=0.25):
    
    # get random sample 
    test = df.sample(frac=frac, axis=0)

    # get everything but the test sample
    train = df.drop(index=test.index)

    return train, test

df = open_csv_clustered()

df_data, df_test = train_test_split(df)

formatted_data = df_data.values.tolist()
formatted_test = df_test.values.tolist()
jobs, skills = list(zip(*formatted_test))
skills = [ast.literal_eval(skill) for skill in skills]

print(len(skills))

option = 2
#Doc2Vec
if option == 1:
    model = trainDoc2Vec(formatted_data)
    correct = 0
    top3 = 0
    for i in range(len(jobs)):
        infer_vector = model.infer_vector(skills[i])
        similar_jobs = model.dv.most_similar([infer_vector], topn=3)
        if(similar_jobs[0][0] == jobs[i]):
            correct = correct + 1
            top3 = top3 + 1
        elif(any(item[0] == jobs[i] for item in similar_jobs)):
            top3 = top3 + 1
        resultCorrect = correct / len(jobs)
        resultTop3 = top3 / len(jobs)
        print("Option 1: Percent of correct guesses: " + str(resultCorrect) + ", percent of top 3 guesses: " + str(resultTop3))

#Word2Vec
elif option == 2:
    job_averages, keys, values, model = prepareOption2(formatted_data)
    correct = 0
    top3 = 0
    for i in range(len(jobs)):
        avg = averageOfSkills(model, skills[i])
        distVec = cdist([avg,], values)
        #sortedList = sorted(list(zip(distVec.transpose(), keys)))
        #topJobs = [i for scor,i in sortedList[0:5]]
        topJobs = [k for dist, k in heapq.nsmallest(3, zip(distVec.transpose(), keys))]
        if(topJobs[0] == jobs[i]):
            correct = correct + 1
            top3 = top3 + 1
        elif(jobs[i] in topJobs):
            top3 = top3 + 1
    resultCorrect = correct / len(jobs)
    resultTop3 = top3 / len(jobs)
    print("Option 2: Rate of correct guesses: " + str(resultCorrect) + ", rate of top 3 guesses: " + str(resultTop3))
elif option == 3:
    job_averages, keys, values, model = prepareOption3(formatted_data)
    correct = 0
    top3 = 0
    for i in range(len(jobs)):
        avg = averageOfSkills(model, skills[i])
        distVec = cdist([avg,], values)
        #sortedList = sorted(list(zip(distVec.transpose(), keys)))
        #topJobs = [i for scor,i in sortedList[0:5]]
        topJobs = [k for dist, k in heapq.nsmallest(3, zip(distVec.transpose(), keys))]
        if(topJobs[0] == jobs[i]):
            correct = correct + 1
            top3 = top3 + 1
        elif(jobs[i] in topJobs):
            top3 = top3 + 1
    resultCorrect = correct / len(jobs)
    resultTop3 = top3 / len(jobs)
    print("Option 3: Rate of correct guesses: " + str(resultCorrect) + ", rate of top 3 guesses: " + str(resultTop3))
'''

'\n# Proper testing\nimport heapq\n\n# divide data into 75 train, 25 test\ndef train_test_split(df, frac=0.25):\n    \n    # get random sample \n    test = df.sample(frac=frac, axis=0)\n\n    # get everything but the test sample\n    train = df.drop(index=test.index)\n\n    return train, test\n\ndf = open_csv_clustered()\n\ndf_data, df_test = train_test_split(df)\n\nformatted_data = df_data.values.tolist()\nformatted_test = df_test.values.tolist()\njobs, skills = list(zip(*formatted_test))\nskills = [ast.literal_eval(skill) for skill in skills]\n\nprint(len(skills))\n\noption = 2\n#Doc2Vec\nif option == 1:\n    model = trainDoc2Vec(formatted_data)\n    correct = 0\n    top3 = 0\n    for i in range(len(jobs)):\n        infer_vector = model.infer_vector(skills[i])\n        similar_jobs = model.dv.most_similar([infer_vector], topn=3)\n        if(similar_jobs[0][0] == jobs[i]):\n            correct = correct + 1\n            top3 = top3 + 1\n        elif(any(item[0] == jobs[i] for item in 

In [14]:
# evaluation and assessment of binary classification models

'''
In the following, I will try to calculate:
•	Condition positive (P): The number of real positive cases in the data.
•	Condition negative (N): The number of real negative cases in the data.
•	True positive (TP): A test result that correctly indicates the presence of a condition or characteristic.
•	True negative (TN): A test result that correctly indicates the absence of a condition or characteristic.
•	False positive (FP): A test result that wrongly indicates that a particular condition or attribute is present.
•	False negative (FN): A test result that wrongly indicates that a particular condition or attribute is absent.
•	Sensitivity, recall, hit rate, or true positive rate (TPR): The proportion of true positives out of all actual positive cases.
•	Specificity, selectivity, or true negative rate (TNR): The proportion of true negatives out of all actual negative cases.
•	Precision or positive predictive value (PPV): The proportion of true positives out of all positive predictions made by the model.
•	Negative predictive value (NPV): The proportion of true negatives out of all negative predictions made by the model.
•	Miss rate or false negative rate (FNR): The proportion of false negatives out of all actual positive cases.
•	Fall-out or false positive rate (FPR): The proportion of false positives out of all actual negative cases.
•	False discovery rate (FDR): The proportion of false positives out of all positive predictions made by the model.
•	False omission rate (FOR): The proportion of false negatives out of all negative predictions made by the model.
•	Positive likelihood ratio (LR+): The ratio of true positive rate to false positive rate.
•	Negative likelihood ratio (LR-): The ratio of false negative rate to true negative rate.
•	Prevalence threshold (PT): A threshold value that balances sensitivity and specificity.
•	Threat score (TS) or critical success index (CSI): The proportion of true positives out of all positive and negative predictions made by the model.
•	Prevalence: The proportion of positive cases in the data.
•	Accuracy (ACC): The proportion of correct predictions (both true positives and true negatives) out of all predictions made by the model.
•	Balanced accuracy (BA): The average of sensitivity and specificity.
•	F1 score: The harmonic mean of precision and sensitivity, providing a single metric to balance both.
•	Phi coefficient (φ) or Matthews correlation coefficient (MCC): A correlation coefficient that takes into account true positives, true negatives, false positives, and false negatives.
•	Fowlkes-Mallows index (FM): A geometric mean of precision and sensitivity.
•	Informedness or bookmaker informedness (BM): The sum of true positive rate and true negative rate, minus 1.
•	Markedness (MK) or deltaP (Δp): The sum of positive predictive value and negative predictive value, minus 1.

'''

'\nIn the following, I will try to calculate:\n•\tCondition positive (P): The number of real positive cases in the data.\n•\tCondition negative (N): The number of real negative cases in the data.\n•\tTrue positive (TP): A test result that correctly indicates the presence of a condition or characteristic.\n•\tTrue negative (TN): A test result that correctly indicates the absence of a condition or characteristic.\n•\tFalse positive (FP): A test result that wrongly indicates that a particular condition or attribute is present.\n•\tFalse negative (FN): A test result that wrongly indicates that a particular condition or attribute is absent.\n•\tSensitivity, recall, hit rate, or true positive rate (TPR): The proportion of true positives out of all actual positive cases.\n•\tSpecificity, selectivity, or true negative rate (TNR): The proportion of true negatives out of all actual negative cases.\n•\tPrecision or positive predictive value (PPV): The proportion of true positives out of all posit

In [15]:
# MODEL EVALUATION 

'''
# Condition positive (P) for Option 1
data = open_csv_clustered()
true_job_titles = data['lda_topic'].tolist()

condition_positive = 0
for job, _ in similar_jobs:
    if job in true_job_titles:
        condition_positive += 1
print("Option 1/ Condition positive (P) -> The number of real positive cases in the data:", condition_positive)

# Condition positive (P) for Option 2
condition_positive = 0
for i in range(len(jobs)):
    avg = averageOfSkills(model, skills[i])
    distVec = cdist([avg,], values)
    topJobs = [k for dist, k in heapq.nsmallest(3, zip(distVec.transpose(), keys))]
    if topJobs[0] == jobs[i]:
        condition_positive += 1
print("Option 2/ Condition positive (P) -> The number of real positive cases in the data:", condition_positive)

'''

'\n# Condition positive (P) for Option 1\ndata = open_csv_clustered()\ntrue_job_titles = data[\'lda_topic\'].tolist()\n\ncondition_positive = 0\nfor job, _ in similar_jobs:\n    if job in true_job_titles:\n        condition_positive += 1\nprint("Option 1/ Condition positive (P) -> The number of real positive cases in the data:", condition_positive)\n\n# Condition positive (P) for Option 2\ncondition_positive = 0\nfor i in range(len(jobs)):\n    avg = averageOfSkills(model, skills[i])\n    distVec = cdist([avg,], values)\n    topJobs = [k for dist, k in heapq.nsmallest(3, zip(distVec.transpose(), keys))]\n    if topJobs[0] == jobs[i]:\n        condition_positive += 1\nprint("Option 2/ Condition positive (P) -> The number of real positive cases in the data:", condition_positive)\n\n'

In [16]:
#Training Part

# Proper testing
import heapq

# divide data into 75 train, 25 test
def train_test_split(df, frac=0.25):
    # get random sample 
    test = df.sample(frac=frac, axis=0)
    # get everything but the test sample
    train = df.drop(index=test.index)

    return train, test

df = open_csv_clustered()

df_data, df_test = train_test_split(df)

formatted_data = df_data.values.tolist()
formatted_test = df_test.values.tolist()
jobs, skills = list(zip(*formatted_test))
skills = [ast.literal_eval(skill) for skill in skills]

#Option 1: Doc2Vec komplett
model = trainDoc2Vec(formatted_data)
# model.save("doc2vec_model.bin")
for i in range(len(jobs)):
    infer_vector = model.infer_vector(skills[i])
    similar_jobs = model.dv.most_similar([infer_vector], topn=3)

#Option 2: Word2Vec komplett
job_averages, keys, values, model = prepareOption2(formatted_data)
correct = 0
top3 = 0
for i in range(len(jobs)):
    avg = averageOfSkills(model, skills[i])
    distVec = cdist([avg,], values)
    #sortedList = sorted(list(zip(distVec.transpose(), keys)))
    #topJobs = [i for scor,i in sortedList[0:5]]
    topJobs = [k for dist, k in heapq.nsmallest(3, zip(distVec.transpose(), keys))]

# Option 3: Doc2Vec Embedding 
job_averages, keys, values, model = prepareOption3(formatted_data)
correct = 0
top3 = 0
for i in range(len(jobs)):
    avg = averageOfSkills(model, skills[i])
    distVec = cdist([avg,], values)
    #sortedList = sorted(list(zip(distVec.transpose(), keys)))
    #topJobs = [i for scor,i in sortedList[0:5]]
    topJobs = [k for dist, k in heapq.nsmallest(3, zip(distVec.transpose(), keys))]

[('text_mining', 0.9873502254486084), ('image_processing', 0.9659996032714844), ('computer_vision', 0.9562139511108398), ('logistic_regression', 0.9512531161308289), ('deep_learning', 0.9502419233322144)]
0.7980417907238007
0.41370654106140137
0.473969042301178
0.5525058507919312
[('image_processing', 0.9678088426589966), ('computer_vision', 0.9677087664604187), ('natural_language_processing', 0.9595919251441956), ('text_mining', 0.9250181317329407), ('sensors', 0.9205660223960876)]
0.6411885619163513
0.44274401664733887
0.3136270046234131
0.9719960577785969


In [17]:
# # from gensim.models import Word2Vec, Doc2Vec
# # from gensim.models.doc2vec import TaggedDocument
# # from scipy.spatial.distance import cdist
# # import ast
# # import numpy as np

# # Function to calculate the average vector of a list of skills
# def average_of_skills(model, input_skills):
#     vectors = [model.wv[skill] for skill in input_skills if skill in model.wv]
#     if len(vectors) == 0:
#         return np.zeros(model.vector_size)
#     return np.average(vectors, axis=0)

# # Option 1: Doc2Vec
# def calculate_similarity_score_option1(model, input_skills, true_skills):
#     input_vector = average_of_skills(model, input_skills)
#     true_skill_vectors = [average_of_skills(model, skills) for skills in true_skills]
#     similarity_scores = [1 - cdist([input_vector], [true_skill_vector], metric='cosine')[0][0] for true_skill_vector in true_skill_vectors]
#     return max(similarity_scores)

# # Option 2: Word2Vec
# def calculate_similarity_score_option2(model, input_skills, true_skills):
#     input_vector = average_of_skills(model, input_skills)
#     true_skill_vectors = [average_of_skills(model, skills) for skills in true_skills]
#     similarity_scores = [1 - cdist([input_vector], [true_skill_vector], metric='cosine')[0][0] for true_skill_vector in true_skill_vectors]
#     return max(similarity_scores)

# # Option 3: Word2Vec
# def calculate_similarity_score_option3(model, input_skills, true_skills):
#     similarity_scores = []
#     for skill in input_skills:
#         top_jobs = model.wv.most_similar([skill], topn=3)
#         job_similarities = [similarity for _, similarity in top_jobs]
#         similarity_scores.append(max(job_similarities))
#     return max(similarity_scores)

# # Load the Doc2Vec model
# model_option1 = Doc2Vec.load("doc2vec_model.bin")  # Load Option 1 model

# # Load the Word2Vec models
# model_option2 = Word2Vec.load("word2vec_model_option2.bin")  # Load Option 2 model
# model_option3 = Word2Vec.load("doc2vec_model_option3.bin")  # Load Option 3 model

# # Load the data
# data = open_csv_clustered()  # Load data
# input_skills = ["skill1", "skill2", "skill3"]  # Example input skills
# true_skills = [["skill4", "skill5", "skill6"], ["skill7", "skill8", "skill9"]]  # Example true skills

# # Calculate similarity score for Option 1 (Doc2Vec)
# similarity_score_option1 = calculate_similarity_score_option1(model_option1, input_skills, true_skills)
# print("Option 1: Similarity Score:", similarity_score_option1)

# # Calculate similarity score for Option 2 (Word2Vec)
# similarity_score_option2 = calculate_similarity_score_option2(model_option2, input_skills, true_skills)
# print("Option 2: Similarity Score:", similarity_score_option2)

# # Calculate similarity score for Option 3 (Word2Vec)
# similarity_score_option3 = calculate_similarity_score_option3(model_option3, input_skills, true_skills)
# print("Option 3: Similarity Score:", similarity_score_option3)


In [32]:
def evaluate_accuracy(predictions, labels):
    correct = 0
    total = len(predictions)
    
    for pred, label in zip(predictions, labels):
        if pred == label:
            correct += 1
    
    accuracy = correct / total
    return accuracy

# Given set of skills
#skills = ["Python, SQL, Machine Learning", "Java, JavaScript, Web Development"]
skills = ["Python, SQL, Machine Learning"]


# Option 1: Doc2Vec komplett
model1 = trainDoc2Vec(formatted_data)
predictions1 = []

for skills in skills:
    infer_vector = model1.infer_vector(skills.split(", "))  # Split skills into a list
    similar_jobs = model1.dv.most_similar([infer_vector], topn=1)
    predicted_job = similar_jobs[0][0]
    predictions1.append(predicted_job)

accuracy1 = evaluate_accuracy(predictions1, jobs)

# Option 2: Word2Vec komplett
job_averages, keys, values, model2 = prepareOption2(formatted_data)
predictions2 = []

for skills in skills:
    avg = averageOfSkills(model2, skills.split(", "))  # Split skills into a list
    distVec = cdist([avg], values)
    topJobs = [k for dist, k in heapq.nsmallest(1, zip(distVec.transpose(), keys))]
    predicted_job = topJobs[0]
    predictions2.append(predicted_job)

accuracy2 = evaluate_accuracy(predictions2, jobs)

# Option 3: Doc2Vec Embedding
job_averages, keys, values, model3 = prepareOption3(formatted_data)
predictions3 = []

for skills in skills:
    avg = averageOfSkills(model3, skills.split(", "))  # Split skills into a list
    distVec = cdist([avg], values)
    topJobs = [k for dist, k in heapq.nsmallest(1, zip(distVec.transpose(), keys))]
    predicted_job = topJobs[0]
    predictions3.append(predicted_job)

accuracy3 = evaluate_accuracy(predictions3, jobs)

# Compare accuracies
print("Accuracy - Option 1: Doc2Vec komplett:", accuracy1)
print("Accuracy - Option 2: Word2Vec komplett:", accuracy2)
print("Accuracy - Option 3: Doc2Vec Embedding:", accuracy3)


[('text_mining', 0.980698823928833), ('computer_vision', 0.9587388038635254), ('deep_learning', 0.9560656547546387), ('logistic_regression', 0.9485481381416321), ('image_processing', 0.9439621567726135)]
0.8279450237751007
0.4098643660545349
0.44455498456954956
0.5119906067848206
[('computer_vision', 0.9666356444358826), ('natural_language_processing', 0.9600709676742554), ('text_mining', 0.95400470495224), ('image_processing', 0.9429821968078613), ('3d', 0.9346281886100769)]
0.6396596729755402
0.3815299868583679
0.3319798707962036
1.0083295674994588
Accuracy - Option 1: Doc2Vec komplett: 0.0
Accuracy - Option 2: Word2Vec komplett: 0.0
Accuracy - Option 3: Doc2Vec Embedding: 0.0


In [35]:
# Given dataset
dataset = [
    ("Research Data Scientist", ['machine_learning', 'python', 'it_skills', 'software_development', 'data_science', 'cloud', 'logistic_regression', 'nosql']),
    ("GAMMA Lead Data Scientist", ['data_analysis', 'operations_research', 'data_science', 'data_management', 'business_analytics', 'consulting', 'management_consulting', 'business_strategy']),
    ("Staff Engineer Data Scientist", ['computer_science', 'text_mining', 'data_analysis', 'machine_learning', 'shell_scripting', 'oracle', 'forecasting', 'ieee'])
]

# Initialize lists to store true labels and predictions
true_labels = []
predictions1 = []
predictions2 = []
predictions3 = []

# Iterate over the dataset
for job, skills in dataset:
    # Append the true label to the true_labels list
    true_labels.append(job)

    # Perform predictions for each option and append to respective prediction lists
    infer_vector = model1.infer_vector(skills)
    similar_jobs = model1.dv.most_similar([infer_vector], topn=1)
    predicted_job = similar_jobs[0][0]
    predictions1.append(predicted_job)

    avg = averageOfSkills(model2, skills)
    distVec = cdist([avg], values)
    topJobs = [k for dist, k in heapq.nsmallest(1, zip(distVec.transpose(), keys))]
    predicted_job = topJobs[0]
    predictions2.append(predicted_job)

    avg = averageOfSkills(model3, skills)
    distVec = cdist([avg], values)
    topJobs = [k for dist, k in heapq.nsmallest(1, zip(distVec.transpose(), keys))]
    predicted_job = topJobs[0]
    predictions3.append(predicted_job)

# Create the confusion matrices
confusion_matrix1 = confusion_matrix(true_labels, predictions1)
confusion_matrix2 = confusion_matrix(true_labels, predictions2)
confusion_matrix3 = confusion_matrix(true_labels, predictions3)


ValueError: XA and XB must have the same number of columns (i.e. feature dimension.)